In [1]:
%load_ext autoreload
%autoreload 2

%autosave 10

#%load_ext lab_black

Autosaving every 10 seconds


In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join("..")))

## Imports

In [3]:
from lfp_analysis.data import *

In [4]:
import numpy as np
import pandas as pd
import h5py

from pathlib import Path

import matplotlib.pyplot as plt

%matplotlib widget
figsize=(8,3)

In [5]:
def parse_time(timestamp):
    assert ('.' in timestamp) and (':' in timestamp), 'Wrong timestamp format, does not contain both ":" and "."'
    parsed_milli = float(timestamp.split('.')[1]) / 1000
    parsed_secs = timestamp.split('.')[0].split(':')
    return float(parsed_secs[0])*60 + float(parsed_secs[1]) + parsed_milli


In [8]:
def regions_to_array(regions_df, label_corrected):
    
    regions_df['Start'] = regions_df['Start'].apply(parse_time)
    regions_df['End'] = regions_df['End'].apply(parse_time)
    
    for ii,row in regions_df.iterrows():
        label_corrected[int(row['Start'] * 2048.0) : int(row['End'] * 2048.0)] = 1
        
    return label_corrected
        

## Read label file

In [12]:
PAT_ID = Path('ET7')

task = 'Pegboard'
stim_cond = 'on'

DATASET_NAME = Path(task+'_'+stim_cond)

DATA_PATH = Path("./../data")
H5_FILE = DATA_PATH / 'preprocessed' / PAT_ID / DATASET_NAME.with_suffix('.h5')

CSV_PATH = DATA_PATH / 'preprocessed' / PAT_ID / Path(str(DATASET_NAME)+'_'+'label') / 'regions_corrected.csv'

In [13]:
h5 = h5py.File(H5_FILE, "r")
LFPs = h5["LFP"][:]
h5.close()


In [14]:
LFPs.shape

(3, 1561039)

In [328]:
regions_df = pd.read_csv(CSV_PATH)

In [329]:
regions_df.head()

,#,Name,Start,End,Length
0,R4,2,0:39.000,1:31.000,0:52.000
1,R6,4,1:47.910,2:25.042,0:37.131
2,R7,5,2:51.000,3:28.000,0:37.000
3,R10,8,3:49.000,4:28.312,0:39.312
4,R11,9,4:55.751,5:29.000,0:33.248


In [330]:
label_corrected = regions_to_array(regions_df, label_corrected);

In [331]:
fig,ax = plt.subplots()
ax.plot(label)
ax.plot(label_corrected*0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [332]:
plt.close('all')

# Overwrite label in H5:

In [333]:
h5 = h5py.File(H5_FILE, "r+")
h5_label = h5["label"]
h5_label[...] = label_corrected
h5.close()